In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import csv
import ffmpeg
import math

import torch
from torch import nn
import torch.nn.functional as F
from torchcodec.decoders import VideoDecoder
from torchvision.transforms import Compose, Resize, Normalize, ToTensor

from skimage.transform import resize

import matplotlib.pyplot as plt

import torch

from src.reader.kinetics import KineticDataset

from src.vqgan import ViTVQGAN

In [ ]:
path = "E:/kinetics-dataset/k400"
split = "train"

ds = KineticDataset(
    path, split,
    n_frames=128,
)


In [ ]:
for i, (token, label) in enumerate(ds):
	print(token.shape)
	
	break

In [ ]:
vitvq = ViTVQGAN(
    image_size=256, patch_size=8,
    dim=768, depth=12, heads=12,
    n_codes=8192, embed_dim=32
)
vitvq.init_from_ckpt("./checkpoint/imagenet_vitvq_base.ckpt")
vitvq.eval()
vitvq.cuda()
print()

In [ ]:

with torch.no_grad():
	for i, (token, label) in enumerate(ds):
		print(token.shape)
		for j in range(token.shape[1]):

			output = vitvq.decode_codes(torch.tensor(token[j]).unsqueeze(0).cuda())
			output_image = output.squeeze(0).permute(1, 2, 0).cpu().numpy()
			output_image = output_image.clip(0, 1)
		
			plt.imshow(output_image)
			plt.axis("off")
			plt.show()

		if i == 10: break


In [ ]:
plt.imshow(output_image)
plt.axis("off")
plt.show()

In [ ]:
# double check the reconstruction loss